# 유아의류 추천시스템

In [136]:
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import surprise

In [346]:
user_sex = input('유저성별')
user_cat = input('유저카테고리')
user_usage = input('유저용도')

유저성별공용
유저카테고리플리스
유저용도유아공동의류


In [396]:
user_select_df = pd.DataFrame(np.array([[user_sex, user_cat, user_usage]]))
user_select_df.columns = ['성별', '카테고리', '용도']
user_select_df

,성별,카테고리,용도
0,공용,플리스,유아공동의류


In [397]:
user_select_df = user_select_df.reset_index()
user_select_df.rename(columns={'index':'상품명'}, inplace=True)
user_select_df['상품명'] = 'user_preference'
user_select_df

,상품명,성별,카테고리,용도
0,user_preference,공용,플리스,유아공동의류


In [398]:
product_df_org = pd.read_excel('../data/추천시스템용 유아복데이터.xlsx')
product_df=product_df_org[['상품명','성별','카테고리','용도']]
product_df.head(3)

,상품명,성별,카테고리,용도
0,[팬콧키즈]20%+10% 쿠폰 팬콧 주니어 윈터 아우터 기모 빅세일,공용,맨투맨,유아동공동의류
1,[페리미츠]레노마키즈 페리미츠 겨울 아우터/맨투맨/팬츠外 70종,여성,자켓,여아의류
2,[버블트리]주니어 여아 의류 아동복 원피스 팬츠 티셔츠,여성,원피스,여아의류


In [399]:
product_df['카테고리'].unique()

array(['맨투맨', '자켓', '원피스', '트레이닝복', '내의', '후드티', '플리스', '티셔츠', '수면조끼',
       '패딩', '면바지'], dtype=object)

In [400]:
product_df['용도'].unique()

array(['유아동공동의류', '여아의류', '유아동스포츠의류', '유아동내의/잠옷', '남아의류', '신생아의류', '여성의류',
       '유아공동의류'], dtype=object)

In [401]:
product_df = product_df.append(user_select_df).reset_index()
product_df.drop('index', axis=1)

,상품명,성별,카테고리,용도
0,[팬콧키즈]20%+10% 쿠폰 팬콧 주니어 윈터 아우터 기모 빅세일,공용,맨투맨,유아동공동의류
1,[페리미츠]레노마키즈 페리미츠 겨울 아우터/맨투맨/팬츠外 70종,여성,자켓,여아의류
2,[버블트리]주니어 여아 의류 아동복 원피스 팬츠 티셔츠,여성,원피스,여아의류
3,[젤리스푼]아동복/아동의류/키즈경량패딩/레깅스/원피스/뽀글이,공용,트레이닝복,유아동스포츠의류
4,[휠라키즈](신세계강남점)휠라키즈 글로브 포켓맨투맨 2종,공용,맨투맨,유아동공동의류
...,...,...,...,...
96,[블루독베이비]크리스마스 수트 SET 40C70-675-03-BON,여성,맨투맨,여아의류
97,[바브키즈]바브키즈 가을신상 티/팬츠 모음전,공용,맨투맨,남아의류
98,[블랙야크키즈]BKS이센스맨투맨티셔츠,공용,티셔츠,유아공동의류
99,[팬콧키즈]팬콧키즈 아동 가을신상 가격인하 50종 택1,여성,원피스,여아의류


In [402]:
product_df_ohc=pd.get_dummies(data=product_df,columns=['성별'],prefix='성별')
product_df_ohc=pd.get_dummies(data=product_df_ohc,columns=['카테고리'],prefix='카테고리')
product_df_ohc=pd.get_dummies(data=product_df_ohc,columns=['용도'],prefix='용도')
product_df.tail()

,index,상품명,성별,카테고리,용도
96,96,[블루독베이비]크리스마스 수트 SET 40C70-675-03-BON,여성,맨투맨,여아의류
97,97,[바브키즈]바브키즈 가을신상 티/팬츠 모음전,공용,맨투맨,남아의류
98,98,[블랙야크키즈]BKS이센스맨투맨티셔츠,공용,티셔츠,유아공동의류
99,99,[팬콧키즈]팬콧키즈 아동 가을신상 가격인하 50종 택1,여성,원피스,여아의류
100,0,user_preference,공용,플리스,유아공동의류


In [403]:
product_df_ohc.drop('index', axis=1, inplace=True)

In [404]:
from numpy import dot
from numpy.linalg import norm
import numpy as np

def cos_sim(A, B):
    result = dot(A, B)/(norm(A)*norm(B))
    return result

In [405]:
product_df_ohc.set_index(['상품명'],inplace=True)

In [406]:
product_df_ohc.values

array([[1, 0, 0, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [407]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(product_df_ohc, product_df_ohc)

# 코사인 유사도가 1이 초과하는 경우로 인한 /3 처리
cosine_sim = cosine_sim/3

In [408]:
cosine_sim

array([[1.        , 0.        , 0.        , ..., 0.33333333, 0.        ,
        0.33333333],
       [0.        , 1.        , 0.66666667, ..., 0.        , 0.66666667,
        0.        ],
       [0.        , 0.66666667, 1.        , ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.33333333, 0.        , 0.        , ..., 1.        , 0.        ,
        0.66666667],
       [0.        , 0.66666667, 1.        , ..., 0.        , 1.        ,
        0.        ],
       [0.33333333, 0.        , 0.        , ..., 0.66666667, 0.        ,
        1.        ]])

In [424]:
indices =pd.Series(range(len(product_df_ohc)),index=product_df_ohc.index)
print(indices.head())

상품명
[팬콧키즈]20%+10% 쿠폰 팬콧 주니어 윈터 아우터 기모 빅세일    0
[페리미츠]레노마키즈 페리미츠 겨울 아우터/맨투맨/팬츠外 70종      1
[버블트리]주니어 여아 의류 아동복 원피스 팬츠 티셔츠           2
[젤리스푼]아동복/아동의류/키즈경량패딩/레깅스/원피스/뽀글이        3
[휠라키즈](신세계강남점)휠라키즈 글로브 포켓맨투맨 2종          4
dtype: int64


In [425]:
indices1=pd.Series(product_df_ohc.index)
print(indices1.head())

0    [팬콧키즈]20%+10% 쿠폰 팬콧 주니어 윈터 아우터 기모 빅세일
1      [페리미츠]레노마키즈 페리미츠 겨울 아우터/맨투맨/팬츠外 70종
2           [버블트리]주니어 여아 의류 아동복 원피스 팬츠 티셔츠
3        [젤리스푼]아동복/아동의류/키즈경량패딩/레깅스/원피스/뽀글이
4          [휠라키즈](신세계강남점)휠라키즈 글로브 포켓맨투맨 2종
Name: 상품명, dtype: object


In [426]:
product_df = product_df.reset_index()
product_df.drop('index', axis=1, inplace=True)

In [506]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 영화를 가지고 연산할 수 있습니다.
    idx = indices[title]

    # 모든 영화에 대해서 해당 상품과의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 상품들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 상품을 받아옵니다.
    sim_scores = sim_scores[1:11]
    
    # 가장 유사한 10개의 상품의 인덱스를 받아옵니다.
    item_indices = [i[0] for i in sim_scores]
    sim_scores_for_df = [i[1] for i in sim_scores]
    print(sim_scores_for_df)
    
    result=pd.DataFrame(indices1[item_indices])
    laster=pd.merge(result,product_df_org,how='left' )
    
    # user_preference가 출력되는 오류 방지
    if laster['상품명'].str.contains('user_preference').any():
        print('start condition code')
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:12]
        item_indices = [i[0] for i in sim_scores]
        sim_scores_for_df = [i[1] for i in sim_scores]
        result=pd.DataFrame(indices1[item_indices])
        laster=pd.merge(result,product_df_org,how='left')
        
    laster = laster[['상품명', '이미지링크', '정상가', '할인가']]
    sim_socres_df = pd.DataFrame({'유사도':sim_scores_for_df})
    sim_socres_df = sim_socres_df['유사도'].round(1)
    result = pd.concat([laster, sim_socres_df], axis=1)
    
    # user_preference 삭제
    if (result['상품명'] == 'user_preference').any():
        result = result[result['상품명'] != 'user_preference']
        
    # 가장 유사한 10개의 상품명 리턴합니다.
    return result

In [508]:
B = get_recommendations('user_preference')
B

[1.0, 1.0, 1.0, 1.0, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666]
start condition code


,상품명,이미지링크,정상가,할인가,유사도
0,[아이더](신세계센텀점)아이더 [21FW] 신상품 몽크 키즈 플리스 자켓 / JUU...,http://gdimg.gmarket.co.kr/2185024493/still/28...,"45,000원","37,350원",1.0
1,[노스페이스키즈]키즈 컴피 EX 플리스 자켓 NJ4FM60 3종 택1,http://gdimg.gmarket.co.kr/2241720630/still/28...,NaN,"65,280원",1.0
2,[휠라키즈][휠라키즈] 역시즌특가/크레마 보아플리스4종택1/FK2PLC4101X,http://gdimg.gmarket.co.kr/1992877145/still/28...,"34,300원","26,070원",1.0
4,[네파키즈]파일 플리스 자켓 KHF6151,http://gdimg.gmarket.co.kr/2259643283/still/30...,"39,000원","30,030원",0.7
5,[네파키즈](신세계강남점)[네파키즈] 21FW 보아 플리스 자켓 KHF6153,http://gdimg.gmarket.co.kr/2177226387/still/30...,"59,000원","43,660원",0.7
6,[네파키즈]리버서블 보아 플리스 자켓 KHF6103,http://gdimg.gmarket.co.kr/2224110884/still/30...,"87,200원","67,150원",0.7
7,[디스커버리] 겨울준비끝 겨울아우터_중계점,http://gdimg.gmarket.co.kr/1894588278/still/60...,NaN,"193,000원",0.7
8,[탑텐키즈](대구신세계)[탑텐키즈] 아동) 덤블보아 롱 집업(MKB4TC3003),http://gdimg.gmarket.co.kr/2165813219/still/30...,"39,900원","35,120원",0.7
9,[네파키즈]보아플리스 베스트 KHF6102,http://gdimg.gmarket.co.kr/2224110885/still/30...,"55,200원","42,510원",0.7
10,[블루독베이비](부산점)블루독베이비 공용 베이비 루돌프 상하복 49D1361006,http://gdimg.gmarket.co.kr/2266496397/still/60...,"31,000원","29,450원",0.7
